In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from root_analysis import find_root_count, find_total_root_length, find_total_root_area, find_root_diameter, find_total_root_volume

In [2]:
def get_image_filenames(directory, recursive=False):
    image_filenames = []

    if recursive:
        for root, _, files in os.walk(directory):
            for filename in files:
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_filenames.append(os.path.join(root, filename))
    else:
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_filenames.append(os.path.join(directory, filename))

    return image_filenames

In [3]:
def calculate_metrics(image: np.ndarray, scaling_factor: float) -> dict:
    root_count = find_root_count(image)

    if root_count == 0:
        return {
            'root_count': 0,
            'root_length': 0,
            'avg_root_length': 0,
            'root_area': 0,
            'avg_root_area': 0,
            'avg_diameter': 0,
            'root_volume': 0,
            'avg_root_volume': 0
        }

    root_length = find_total_root_length(image, scaling_factor)
    avg_root_length = root_length / root_count
    root_area = find_total_root_area(image, scaling_factor)
    avg_root_area = root_area / root_count
    avg_diameter = find_root_diameter(image, scaling_factor)
    root_volume = find_total_root_volume(image, scaling_factor)
    avg_root_volume = root_volume / root_count

    return {
        'root_count': root_count,
        'root_length': root_length,
        'avg_root_length': avg_root_length,
        'root_area': root_area,
        'avg_root_area': avg_root_area,
        'avg_diameter': avg_diameter,
        'root_volume': root_volume,
        'avg_root_volume': avg_root_volume
    }

In [4]:
image_filenames = get_image_filenames('mrphotos/output', recursive=True)

In [5]:
measurements = pd.DataFrame(columns=['image', 'layer', 'root_count', 'root_length',
                            'avg_root_length', 'root_area', 'avg_root_area', 'avg_diameter', 'root_volume', 'avg_root_volume'])

In [6]:
for image_filename in image_filenames:
    tube_lower_end = int(os.path.basename(image_filename).split('_')[2].removeprefix('L').removesuffix('.png').split('-')[0])
    tube_higher_end = int(os.path.basename(image_filename).split('_')[2].removeprefix('L').removesuffix('.png').split('-')[1])
    segments = tube_higher_end - tube_lower_end + 1

    image = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)
    segment_width = image.shape[1] // segments

    for layer in range(tube_lower_end, tube_higher_end + 1):
        segment = image[:, (layer - 1) * segment_width:layer * segment_width]
        metrics = calculate_metrics(segment, 0.2581)

        measurements.loc[len(measurements)] = {
            'image': image_filename,
            'layer': layer,
            **metrics
        }

In [8]:
measurements.to_csv('analysis/layered_measurements.csv', index=False)